# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"C:\Users\charl\Documents\Instruments and calibrations"
DayFolder = os.path.join(MasterFolder, "UCB_6-8-23")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')

output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,06 FDCA-P60-1_CRR_DiadFit,103.548947,0.006325,0.005347,1285.110681,0.005234,1668.092070,1285.110681,3446.752873,...,0.955606,1370.313519,84.245801,0.307644,NaN,NaN,NaN,NaN,NaN,NaN
1,0,07 FDCA-P60-2,103.552475,0.006376,0.005343,1285.106366,0.005216,1675.071443,1285.106366,3470.278110,...,0.941495,1370.644359,83.821480,0.308460,NaN,NaN,NaN,NaN,NaN,NaN
2,0,08 FDCA-P60-3,103.553497,0.006165,0.005187,1285.105751,0.005070,1700.089086,1285.105751,3529.536828,...,0.915880,1370.651406,77.641382,0.309075,NaN,NaN,NaN,NaN,NaN,NaN
3,0,10 FDCA-P59-1,103.611642,0.004099,0.003045,1284.961979,0.002709,1676.812960,1284.962029,3739.647130,...,1.052990,1370.624326,83.270674,0.319117,NaN,NaN,NaN,NaN,NaN,NaN
4,0,11 FDCA-P58-1,103.631029,0.003250,0.002579,1284.916416,0.002452,1825.197392,1284.916466,4118.820976,...,1.018740,1370.309895,99.186909,0.325307,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['06 FDCA-P60-1.txt',
 '07 FDCA-P60-2.txt',
 '08 FDCA-P60-3.txt',
 '10 FDCA-P59-1.txt',
 '11 FDCA-P58-1.txt',
 '12 FDCA-P57-1.txt',
 '14 FDCA-P56-1.txt',
 '15 FDCA-P56-2.txt',
 '16 FDCA-P56-3.txt',
 '18 FDCA-P55-1.txt',
 '20 FDCA-P54-1.txt',
 '22 FDCA-P53-1.txt',
 '24 FDCA-P52-1.txt',
 '25 FDCA-P51-1.txt',
 '27 FDCA-P50-1.txt',
 '28 FDCA-P49-1.txt',
 '30 FDCA-P48-1.txt',
 '31 FDCA-P47-1.txt',
 '33 FDCA-P46-1.txt',
 '34 FDCA-P45-1.txt',
 '36 FDCA-P44-1.txt',
 '38 FDCA-P43-1.txt',
 '39 FDCA-P42-1.txt',
 '40 FDCA-P41-1.txt',
 '42 FDCA-P40-1.txt',
 '44 FDCA-P39-1.txt',
 '46 FDCA-P37-1.txt',
 '47 FDCA-P36-1.txt',
 '48 FDCA-P35-1.txt',
 '49 FDCA-P34-1.txt',
 '51 FDCA-P33-1.txt',
 '52 FDCA-P32-1.txt',
 '53 FDCA-P31-1.txt',
 '55 FDCA-P30-1.txt',
 '56 FDCA-P29-1.txt',
 '58 FDCA-P28-1.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 36/36 [00:00<00:00, 969.35it/s]

Done


'06 FDCA-P60-1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'06 FDCA-P60-1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['06 FDCA-P60-1', '07 FDCA-P60-2', '08 FDCA-P60-3', '10 FDCA-P59-1',
       '11 FDCA-P58-1', '12 FDCA-P57-1', '14 FDCA-P56-1', '15 FDCA-P56-2',
       '16 FDCA-P56-3', '18 FDCA-P55-1', '20 FDCA-P54-1', '22 FDCA-P53-1',
       '24 FDCA-P52-1', '25 FDCA-P51-1', '27 FDCA-P50-1', '28 FDCA-P49-1',
       '30 FDCA-P48-1', '31 FDCA-P47-1', '33 FDCA-P46-1', '34 FDCA-P45-1',
       '36 FDCA-P44-1', '38 FDCA-P43-1', '39 FDCA-P42-1', '40 FDCA-P41-1',
       '42 FDCA-P40-1', '44 FDCA-P39-1', '46 FDCA-P37-1', '47 FDCA-P36-1',
       '48 FDCA-P35-1', '49 FDCA-P34-1', '51 FDCA-P33-1', '52 FDCA-P32-1',
       '53 FDCA-P31-1', '55 FDCA-P30-1', '56 FDCA-P29-1', '58 FDCA-P28-1'],
      dtype='<U13')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


1


,time,preferred_values,lower_values,upper_values
0,44849.0,0.997582,0.997574,0.997589
1,44913.0,0.997582,0.997574,0.997589
2,44965.0,0.997581,0.997574,0.997589
3,45751.0,0.997577,0.997570,0.997585
4,46110.0,0.997576,0.997568,0.997583


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

In [15]:
file_sec_phase

NameError: name 'file_sec_phase' is not defined

In [16]:
df_combo['name_for_matching']

0     06 FDCA-P60-1
1     07 FDCA-P60-2
2     08 FDCA-P60-3
3     10 FDCA-P59-1
4     11 FDCA-P58-1
5     12 FDCA-P57-1
6     14 FDCA-P56-1
7     15 FDCA-P56-2
8     16 FDCA-P56-3
9     18 FDCA-P55-1
10    20 FDCA-P54-1
11    22 FDCA-P53-1
12    24 FDCA-P52-1
13    25 FDCA-P51-1
14    27 FDCA-P50-1
15    28 FDCA-P49-1
16    30 FDCA-P48-1
17    31 FDCA-P47-1
18    33 FDCA-P46-1
19    34 FDCA-P45-1
20    36 FDCA-P44-1
21    38 FDCA-P43-1
22    39 FDCA-P42-1
23    40 FDCA-P41-1
24    42 FDCA-P40-1
25    44 FDCA-P39-1
26    46 FDCA-P37-1
27    47 FDCA-P36-1
28    48 FDCA-P35-1
29    49 FDCA-P34-1
30    51 FDCA-P33-1
31    52 FDCA-P32-1
32    53 FDCA-P31-1
33    55 FDCA-P30-1
34    56 FDCA-P29-1
35    58 FDCA-P28-1
Name: name_for_matching, dtype: object

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,0,103.298551,103.303940,103.293162,0.005389,06 FDCA-P60-1_CRR_DiadFit,103.548947,0.006325,0.005347,1285.110681,...,June,8.0,34.482,10.0,5.0,5.0,"['0h', '0m', '52s']",12:27:29 PM\n,44849.0,1325.004
1,0,103.302037,103.307423,103.296651,0.005386,07 FDCA-P60-2,103.552475,0.006376,0.005343,1285.106366,...,June,8.0,34.474,10.0,5.0,5.0,"['0h', '0m', '51s']",12:28:33 PM\n,44913.0,1325.004
2,0,103.303029,103.308260,103.297797,0.005231,08 FDCA-P60-3,103.553497,0.006165,0.005187,1285.105751,...,June,8.0,34.470,10.0,5.0,5.0,"['0h', '0m', '51s']",12:29:25 PM\n,44965.0,1325.004
3,0,103.360624,103.363757,103.357491,0.003133,10 FDCA-P59-1,103.611642,0.004099,0.003045,1284.961979,...,June,8.0,34.560,10.0,5.0,5.0,"['0h', '0m', '51s']",12:42:31 PM\n,45751.0,1325.004
4,0,103.379783,103.382467,103.377099,0.002684,11 FDCA-P58-1,103.631029,0.003250,0.002579,1284.916416,...,June,8.0,34.491,10.0,5.0,5.0,"['0h', '0m', '51s']",12:48:30 PM\n,46110.0,1325.004


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')


In [19]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()